# **Undirected Weighted Graphs - Complete Guide**

An **Undirected Weighted Graph** is a collection of vertices (nodes) connected by edges where each edge has an associated weight/cost and no direction. This means if there's an edge between vertex A and vertex B with weight W, you can travel from A to B and also from B to A with the same weight W.

**Key Features:**
- **Vertices (Nodes)**: Individual points or entities in the graph
- **Weighted Edges**: Connections between vertices with associated costs/weights
- **Symmetric Relations**: If (u,v) has weight W, then (v,u) also has weight W
- **Degree**: Number of edges connected to a vertex
- **Path Weight**: Sum of all edge weights in a path
- **Shortest Path**: Path with minimum total weight between two vertices

**Graph Terminology:**
- **Adjacent Vertices**: Two vertices connected by an edge
- **Connected Graph**: Path exists between every pair of vertices  
- **Disconnected Graph**: Some vertices are not reachable from others
- **Complete Graph**: Every vertex is connected to every other vertex
- **Spanning Tree**: Subgraph that connects all vertices with minimum edges (V-1 edges)
- **Minimum Spanning Tree (MST)**: Spanning tree with minimum total weight

**Applications:**
- **Transportation Networks**: Road networks with distances, flight routes with costs
- **Social Networks**: Relationship strength, communication frequency
- **Computer Networks**: Network topology with bandwidth/latency weights
- **Game Development**: Pathfinding with terrain costs, AI decision making
- **Supply Chain**: Distribution networks with shipping costs
- **Biology**: Genetic similarity networks, protein interaction strengths

## Node Definition for Weighted Graphs

A **Node (Vertex)** is the fundamental building block of a weighted graph. Each node represents an entity in the system, and edges between nodes represent weighted relationships or connections.

In weighted graphs, we need to store:
- **Node Identity**: Unique identifier (name, number, etc.)
- **Edge Information**: Connected nodes and their associated weights
- **Traversal State**: Visited status, distances, parent pointers (for algorithms)

We'll represent nodes as simple identifiers and manage their connections through graph data structures that store both adjacency and weight information.

In [2]:
import random
from collections import defaultdict, deque
import heapq
import sys

# Create a sample undirected weighted graph with 7 nodes
nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

# Define edges with weights for our sample graph
# Format: (node1, node2, weight)
edges = [
    ('A', 'B', 4), ('A', 'C', 2),
    ('B', 'C', 1), ('B', 'D', 5),
    ('C', 'D', 8), ('C', 'E', 10),
    ('D', 'E', 2), ('D', 'F', 6),
    ('E', 'F', 3), ('E', 'G', 1),
    ('F', 'G', 4)
]

print("=== Sample Undirected Weighted Graph ===")
print(f"Nodes: {nodes}")
print(f"Edges with weights:")
for u, v, w in edges:
    print(f"  {u} ↔ {v} (weight: {w})")

print(f"\nTotal Nodes: {len(nodes)}")
print(f"Total Edges: {len(edges)}")

=== Sample Undirected Weighted Graph ===
Nodes: ['A', 'B', 'C', 'D', 'E', 'F', 'G']
Edges with weights:
  A ↔ B (weight: 4)
  A ↔ C (weight: 2)
  B ↔ C (weight: 1)
  B ↔ D (weight: 5)
  C ↔ D (weight: 8)
  C ↔ E (weight: 10)
  D ↔ E (weight: 2)
  D ↔ F (weight: 6)
  E ↔ F (weight: 3)
  E ↔ G (weight: 1)
  F ↔ G (weight: 4)

Total Nodes: 7
Total Edges: 11


## Graph Representations

There are two main ways to represent weighted graphs in memory:

### 1. Adjacency Matrix
- **2D array** where `matrix[i][j]` stores the weight of edge between vertices i and j
- **0 or ∞** indicates no edge between vertices
- **Symmetric matrix** for undirected graphs (matrix[i][j] = matrix[j][i])

### 2. Adjacency List  
- **Dictionary/List** where each vertex stores a list of its neighbors with weights
- **Space efficient** for sparse graphs
- **Easy to iterate** through neighbors of a vertex

**Time Complexity Comparison:**

| Operation | Adjacency Matrix | Adjacency List |
|-----------|------------------|----------------|
| Add Edge | O(1) | O(1) |
| Remove Edge | O(1) | O(degree) |
| Check Edge | O(1) | O(degree) |
| Get All Neighbors | O(V) | O(degree) |
| Space Complexity | O(V²) | O(V + E) |

In [3]:
class WeightedGraph:
    def __init__(self, nodes):
        self.nodes = nodes
        self.node_to_index = {node: i for i, node in enumerate(nodes)}
        self.num_nodes = len(nodes)
        
        # Adjacency Matrix representation
        self.adj_matrix = [[0 if i == j else float('inf') for j in range(self.num_nodes)] 
                          for i in range(self.num_nodes)]
        
        # Adjacency List representation
        self.adj_list = defaultdict(list)
    
    def add_edge(self, u, v, weight):
        """Add weighted edge between vertices u and v"""
        u_idx = self.node_to_index[u]
        v_idx = self.node_to_index[v]
        
        # Update adjacency matrix
        self.adj_matrix[u_idx][v_idx] = weight
        self.adj_matrix[v_idx][u_idx] = weight  # Undirected graph
        
        # Update adjacency list
        self.adj_list[u].append((v, weight))
        self.adj_list[v].append((u, weight))  # Undirected graph
    
    def display_matrix(self):
        """Display adjacency matrix representation"""
        print("Adjacency Matrix:")
        print("    ", end="")
        for node in self.nodes:
            print(f"{node:>4}", end="")
        print()
        
        for i, node in enumerate(self.nodes):
            print(f"{node:>2}: ", end="")
            for j in range(self.num_nodes):
                val = self.adj_matrix[i][j]
                if val == float('inf'):
                    print("  ∞", end="")
                else:
                    print(f"{val:>3}", end="")
            print()
    
    def display_list(self):
        """Display adjacency list representation"""
        print("Adjacency List:")
        for node in self.nodes:
            neighbors = [(neighbor, weight) for neighbor, weight in self.adj_list[node]]
            print(f"{node}: {neighbors}")

# Create and populate our sample graph
graph = WeightedGraph(nodes)

# Add all edges to both representations
for u, v, weight in edges:
    graph.add_edge(u, v, weight)

print("=== Graph Representations ===")
graph.display_matrix()
print()
graph.display_list()

=== Graph Representations ===
Adjacency Matrix:
       A   B   C   D   E   F   G
 A:   0  4  2  ∞  ∞  ∞  ∞
 B:   4  0  1  5  ∞  ∞  ∞
 C:   2  1  0  8 10  ∞  ∞
 D:   ∞  5  8  0  2  6  ∞
 E:   ∞  ∞ 10  2  0  3  1
 F:   ∞  ∞  ∞  6  3  0  4
 G:   ∞  ∞  ∞  ∞  1  4  0

Adjacency List:
A: [('B', 4), ('C', 2)]
B: [('A', 4), ('C', 1), ('D', 5)]
C: [('A', 2), ('B', 1), ('D', 8), ('E', 10)]
D: [('B', 5), ('C', 8), ('E', 2), ('F', 6)]
E: [('C', 10), ('D', 2), ('F', 3), ('G', 1)]
F: [('D', 6), ('E', 3), ('G', 4)]
G: [('E', 1), ('F', 4)]


## Breadth-First Search (BFS)

**BFS** explores the graph level by level, visiting all neighbors of a vertex before moving to the next level. In weighted graphs, BFS finds the path with the **minimum number of edges** (not necessarily minimum weight).

**Algorithm Steps:**
1. Start from source vertex, mark it visited
2. Add source to queue
3. While queue is not empty:
   - Dequeue vertex
   - Visit all unvisited neighbors
   - Mark neighbors as visited and enqueue them
4. Process vertices in the order they are dequeued

**Time Complexity:** O(V + E) where V is vertices and E is edges
**Space Complexity:** O(V) for the queue and visited array

**Use Cases:**
- **Shortest hop count** between vertices
- **Level-order traversal** of graph
- **Finding connected components**
- **Bipartite graph detection**

In [4]:
def bfs(graph, start_node):
    """
    Breadth-First Search traversal of the graph
    Returns list of nodes in BFS order and distances from start
    """
    visited = set()
    queue = deque([start_node])
    bfs_order = []
    distances = {start_node: 0}
    parent = {start_node: None}
    
    visited.add(start_node)
    
    print(f"Starting BFS from node: {start_node}")
    print("Step by step traversal:")
    
    step = 1
    while queue:
        current_node = queue.popleft()
        bfs_order.append(current_node)
        
        print(f"Step {step}: Visit {current_node} (distance: {distances[current_node]})")
        
        # Get neighbors from adjacency list
        neighbors = []
        for neighbor, weight in graph.adj_list[current_node]:
            if neighbor not in visited:
                neighbors.append((neighbor, weight))
                visited.add(neighbor)
                queue.append(neighbor)
                distances[neighbor] = distances[current_node] + 1
                parent[neighbor] = current_node
        
        if neighbors:
            neighbor_names = [f"{n}(w:{w})" for n, w in neighbors]
            print(f"         Add to queue: {neighbor_names}")
        
        print(f"         Queue state: {list(queue)}")
        step += 1
        print()
    
    return bfs_order, distances, parent

# Perform BFS traversal
bfs_result, bfs_distances, bfs_parent = bfs(graph, 'A')

print("=== BFS Results ===")
print(f"BFS Order: {' -> '.join(bfs_result)}")
print(f"Distances from A: {bfs_distances}")
print(f"Parent nodes: {bfs_parent}")

Starting BFS from node: A
Step by step traversal:
Step 1: Visit A (distance: 0)
         Add to queue: ['B(w:4)', 'C(w:2)']
         Queue state: ['B', 'C']

Step 2: Visit B (distance: 1)
         Add to queue: ['D(w:5)']
         Queue state: ['C', 'D']

Step 3: Visit C (distance: 1)
         Add to queue: ['E(w:10)']
         Queue state: ['D', 'E']

Step 4: Visit D (distance: 2)
         Add to queue: ['F(w:6)']
         Queue state: ['E', 'F']

Step 5: Visit E (distance: 2)
         Add to queue: ['G(w:1)']
         Queue state: ['F', 'G']

Step 6: Visit F (distance: 3)
         Queue state: ['G']

Step 7: Visit G (distance: 3)
         Queue state: []

=== BFS Results ===
BFS Order: A -> B -> C -> D -> E -> F -> G
Distances from A: {'A': 0, 'B': 1, 'C': 1, 'D': 2, 'E': 2, 'F': 3, 'G': 3}
Parent nodes: {'A': None, 'B': 'A', 'C': 'A', 'D': 'B', 'E': 'C', 'F': 'D', 'G': 'E'}


## Depth-First Search (DFS) - Inorder, Preorder, Postorder

**DFS** explores as far as possible along each branch before backtracking. For graphs, we have three main orderings:

### 1. Preorder DFS
- **Process vertex** before visiting its neighbors
- **Root → Left → Right** analog for trees

### 2. Inorder DFS  
- Not typically used for general graphs
- **Left → Root → Right** analog for trees
- More relevant for binary trees

### 3. Postorder DFS
- **Process vertex** after visiting all its neighbors  
- **Left → Right → Root** analog for trees

**Algorithm Steps:**
1. Mark current vertex as visited
2. For Preorder: Process current vertex
3. Recursively visit all unvisited neighbors
4. For Postorder: Process current vertex after neighbors

**Time Complexity:** O(V + E) 
**Space Complexity:** O(V) for recursion stack and visited array

**Use Cases:**
- **Topological sorting** (Postorder)
- **Cycle detection**
- **Connected components**
- **Pathfinding and backtracking**

In [5]:
def dfs_preorder(graph, start_node, visited=None, order=None, step_counter=None):
    """DFS with Preorder traversal (process node before neighbors)"""
    if visited is None:
        visited = set()
        order = []
        step_counter = [1]
        print(f"Starting DFS Preorder from node: {start_node}")
        print("Step by step traversal:")
    
    # Preorder: Process current node first
    visited.add(start_node)
    order.append(start_node)
    
    print(f"Step {step_counter[0]}: Process {start_node} (Preorder)")
    step_counter[0] += 1
    
    # Visit all unvisited neighbors
    neighbors = [neighbor for neighbor, weight in graph.adj_list[start_node]]
    for neighbor in sorted(neighbors):  # Sort for consistent output
        if neighbor not in visited:
            print(f"         Recursively visit {neighbor}")
            dfs_preorder(graph, neighbor, visited, order, step_counter)
    
    return order

def dfs_postorder(graph, start_node, visited=None, order=None, step_counter=None):
    """DFS with Postorder traversal (process node after neighbors)"""
    if visited is None:
        visited = set()
        order = []
        step_counter = [1]
        print(f"Starting DFS Postorder from node: {start_node}")
        print("Step by step traversal:")
    
    visited.add(start_node)
    
    # Visit all unvisited neighbors first
    neighbors = [neighbor for neighbor, weight in graph.adj_list[start_node]]
    for neighbor in sorted(neighbors):
        if neighbor not in visited:
            print(f"Step {step_counter[0]}: Recursively visit {neighbor}")
            step_counter[0] += 1
            dfs_postorder(graph, neighbor, visited, order, step_counter)
    
    # Postorder: Process current node after all neighbors
    order.append(start_node)
    print(f"Step {step_counter[0]}: Process {start_node} (Postorder)")
    step_counter[0] += 1
    
    return order

def dfs_inorder_simulation(graph, start_node):
    """
    Simulated Inorder for graphs (not standard, but showing concept)
    Process node between visiting left and right neighbors
    """
    visited = set()
    order = []
    
    def dfs_helper(node):
        visited.add(node)
        neighbors = sorted([neighbor for neighbor, weight in graph.adj_list[node]])
        
        # Visit first half of neighbors (simulating "left")
        mid = len(neighbors) // 2
        for i in range(mid):
            if neighbors[i] not in visited:
                dfs_helper(neighbors[i])
        
        # Process current node (simulating "root")
        order.append(node)
        
        # Visit remaining neighbors (simulating "right") 
        for i in range(mid, len(neighbors)):
            if neighbors[i] not in visited:
                dfs_helper(neighbors[i])
    
    print(f"Starting DFS Inorder simulation from node: {start_node}")
    dfs_helper(start_node)
    return order

# Perform all DFS variations
print("=== DFS Preorder ===")
preorder_result = dfs_preorder(graph, 'A')
print(f"Preorder: {' -> '.join(preorder_result)}")
print()

print("=== DFS Postorder ===")
postorder_result = dfs_postorder(graph, 'A')
print(f"Postorder: {' -> '.join(postorder_result)}")
print()

print("=== DFS Inorder (Simulated) ===")
inorder_result = dfs_inorder_simulation(graph, 'A')
print(f"Inorder: {' -> '.join(inorder_result)}")

=== DFS Preorder ===
Starting DFS Preorder from node: A
Step by step traversal:
Step 1: Process A (Preorder)
         Recursively visit B
Step 2: Process B (Preorder)
         Recursively visit C
Step 3: Process C (Preorder)
         Recursively visit D
Step 4: Process D (Preorder)
         Recursively visit E
Step 5: Process E (Preorder)
         Recursively visit F
Step 6: Process F (Preorder)
         Recursively visit G
Step 7: Process G (Preorder)
Preorder: A -> B -> C -> D -> E -> F -> G

=== DFS Postorder ===
Starting DFS Postorder from node: A
Step by step traversal:
Step 1: Recursively visit B
Step 2: Recursively visit C
Step 3: Recursively visit D
Step 4: Recursively visit E
Step 5: Recursively visit F
Step 6: Recursively visit G
Step 7: Process G (Postorder)
Step 8: Process F (Postorder)
Step 9: Process E (Postorder)
Step 10: Process D (Postorder)
Step 11: Process C (Postorder)
Step 12: Process B (Postorder)
Step 13: Process A (Postorder)
Postorder: G -> F -> E -> D -> C -> 

## Dijkstra's Algorithm - Shortest Path

**Dijkstra's Algorithm** finds the shortest paths from a source vertex to all other vertices in a weighted graph with **non-negative edge weights**.

**Algorithm Steps:**
1. Initialize distances to all vertices as infinity, except source (distance 0)
2. Create a min-heap with all vertices
3. While heap is not empty:
   - Extract vertex with minimum distance
   - For each neighbor, if distance through current vertex is shorter, update distance
   - Add updated vertices back to heap
4. Result: shortest distances and paths to all vertices

**Time Complexity:** O((V + E) log V) with binary heap
**Space Complexity:** O(V) for distance and parent arrays

**Key Properties:**
- **Greedy approach**: Always picks the closest unvisited vertex
- **Optimal substructure**: Shortest path contains shortest subpaths
- **Non-negative weights**: Cannot handle negative edge weights

**Use Cases:**
- **GPS Navigation**: Finding shortest routes
- **Network Routing**: Internet packet routing protocols
- **Game AI**: Pathfinding with terrain costs

In [6]:
def dijkstra(graph, start_node):
    """
    Dijkstra's algorithm for finding shortest paths from start_node to all other nodes
    Returns distances and parent pointers for path reconstruction
    """
    # Initialize distances and parent pointers
    distances = {node: float('inf') for node in graph.nodes}
    distances[start_node] = 0
    parent = {node: None for node in graph.nodes}
    visited = set()
    
    # Priority queue: (distance, node)
    pq = [(0, start_node)]
    
    print(f"Starting Dijkstra's algorithm from node: {start_node}")
    print("Step by step execution:")
    
    step = 1
    while pq:
        current_dist, current_node = heapq.heappop(pq)
        
        # Skip if already processed
        if current_node in visited:
            continue
            
        visited.add(current_node)
        
        print(f"Step {step}: Process {current_node} (distance: {current_dist})")
        
        # Check all neighbors
        updates = []
        for neighbor, weight in graph.adj_list[current_node]:
            if neighbor not in visited:
                new_distance = distances[current_node] + weight
                
                if new_distance < distances[neighbor]:
                    distances[neighbor] = new_distance
                    parent[neighbor] = current_node
                    heapq.heappush(pq, (new_distance, neighbor))
                    updates.append(f"{neighbor}: {new_distance}")
        
        if updates:
            print(f"         Updated distances: {', '.join(updates)}")
        
        # Show current priority queue state
        pq_state = [(dist, node) for dist, node in pq if node not in visited]
        if pq_state:
            print(f"         Priority queue: {pq_state}")
        
        step += 1
        print()
    
    return distances, parent

def reconstruct_path(parent, start, end):
    """Reconstruct shortest path from start to end using parent pointers"""
    if parent[end] is None and end != start:
        return []
    
    path = []
    current = end
    while current is not None:
        path.append(current)
        current = parent[current]
    
    return path[::-1]  # Reverse to get path from start to end

# Run Dijkstra's algorithm
dijkstra_distances, dijkstra_parent = dijkstra(graph, 'A')

print("=== Dijkstra's Results ===")
print(f"Shortest distances from A: {dijkstra_distances}")
print(f"Parent pointers: {dijkstra_parent}")

print("\nShortest paths from A to all nodes:")
for node in graph.nodes:
    if node != 'A':
        path = reconstruct_path(dijkstra_parent, 'A', node)
        if path:
            path_weight = dijkstra_distances[node]
            print(f"A → {node}: {' → '.join(path)} (weight: {path_weight})")
        else:
            print(f"A → {node}: No path exists")

Starting Dijkstra's algorithm from node: A
Step by step execution:
Step 1: Process A (distance: 0)
         Updated distances: B: 4, C: 2
         Priority queue: [(2, 'C'), (4, 'B')]

Step 2: Process C (distance: 2)
         Updated distances: B: 3, D: 10, E: 12
         Priority queue: [(3, 'B'), (4, 'B'), (10, 'D'), (12, 'E')]

Step 3: Process B (distance: 3)
         Updated distances: D: 8
         Priority queue: [(8, 'D'), (10, 'D'), (12, 'E')]

Step 4: Process D (distance: 8)
         Updated distances: E: 10, F: 14
         Priority queue: [(12, 'E'), (10, 'E'), (14, 'F')]

Step 5: Process E (distance: 10)
         Updated distances: F: 13, G: 11
         Priority queue: [(11, 'G'), (13, 'F'), (14, 'F')]

Step 6: Process G (distance: 11)
         Priority queue: [(14, 'F'), (13, 'F')]

Step 7: Process F (distance: 13)

=== Dijkstra's Results ===
Shortest distances from A: {'A': 0, 'B': 3, 'C': 2, 'D': 8, 'E': 10, 'F': 13, 'G': 11}
Parent pointers: {'A': None, 'B': 'C', 'C': 'A

## Bellman-Ford Algorithm - Shortest Path with Negative Weights

**Bellman-Ford Algorithm** finds shortest paths from a source vertex to all other vertices, and can handle **negative edge weights**. It also **detects negative cycles**.

**Algorithm Steps:**
1. Initialize distances to all vertices as infinity, except source (distance 0)
2. For (V-1) iterations:
   - For each edge (u,v) with weight w:
     - If distance[u] + w < distance[v], update distance[v] and parent[v]
3. Run one more iteration to detect negative cycles
4. If any distance can still be updated, negative cycle exists

**Time Complexity:** O(V × E) - always, regardless of graph structure
**Space Complexity:** O(V) for distance and parent arrays

**Key Properties:**
- **Dynamic Programming**: Builds up shortest paths iteratively
- **Negative weight handling**: Can process negative edge weights
- **Negative cycle detection**: Identifies if shortest paths are undefined
- **Slower than Dijkstra**: But more versatile

**Use Cases:**
- **Currency arbitrage**: Detecting profitable exchange cycles
- **Network analysis**: Routing with quality metrics (can be negative)
- **Game theory**: Finding optimal strategies with penalties

In [7]:
def bellman_ford(graph, start_node):
    """
    Bellman-Ford algorithm for finding shortest paths with negative weight support
    Returns distances, parent pointers, and whether negative cycle exists
    """
    # Initialize distances and parent pointers
    distances = {node: float('inf') for node in graph.nodes}
    distances[start_node] = 0
    parent = {node: None for node in graph.nodes}
    
    # Convert adjacency list to edge list for easier processing
    edge_list = []
    for u in graph.adj_list:
        for v, weight in graph.adj_list[u]:
            edge_list.append((u, v, weight))
    
    print(f"Starting Bellman-Ford algorithm from node: {start_node}")
    print(f"Edge list: {edge_list}")
    print("Step by step execution:")
    
    # Relax edges V-1 times
    for iteration in range(len(graph.nodes) - 1):
        print(f"\nIteration {iteration + 1}:")
        updated = False
        
        for u, v, weight in edge_list:
            if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                old_distance = distances[v]
                distances[v] = distances[u] + weight
                parent[v] = u
                updated = True
                print(f"  Updated {v}: {old_distance} → {distances[v]} (via {u})")
        
        if not updated:
            print(f"  No updates in iteration {iteration + 1}, algorithm can terminate early")
            break
        
        print(f"  Current distances: {distances}")
    
    # Check for negative cycles
    print(f"\nNegative cycle detection:")
    has_negative_cycle = False
    
    for u, v, weight in edge_list:
        if distances[u] != float('inf') and distances[u] + weight < distances[v]:
            has_negative_cycle = True
            print(f"  Negative cycle detected involving edge {u} → {v}")
            break
    
    if not has_negative_cycle:
        print(f"  No negative cycle detected")
    
    return distances, parent, has_negative_cycle

# Run Bellman-Ford algorithm on our graph
bf_distances, bf_parent, has_neg_cycle = bellman_ford(graph, 'A')

print("\n=== Bellman-Ford Results ===")
print(f"Shortest distances from A: {bf_distances}")
print(f"Parent pointers: {bf_parent}")
print(f"Negative cycle exists: {has_neg_cycle}")

print("\nShortest paths from A to all nodes:")
for node in graph.nodes:
    if node != 'A':
        path = reconstruct_path(bf_parent, 'A', node)
        if path and not has_neg_cycle:
            path_weight = bf_distances[node]
            print(f"A → {node}: {' → '.join(path)} (weight: {path_weight})")
        elif has_neg_cycle:
            print(f"A → {node}: Undefined due to negative cycle")
        else:
            print(f"A → {node}: No path exists")

# Demonstrate with a graph that has negative weights
print("\n=== Testing with Negative Weights ===")
negative_edges = [
    ('A', 'B', 4), ('A', 'C', 2),
    ('B', 'C', -3), ('B', 'D', 5),  # Negative weight edge
    ('C', 'D', 1)
]

neg_graph = WeightedGraph(['A', 'B', 'C', 'D'])
for u, v, w in negative_edges:
    neg_graph.add_edge(u, v, w)

print("Graph with negative weight edge B-C: -3")
neg_distances, neg_parent, neg_cycle = bellman_ford(neg_graph, 'A')
print(f"Results: {neg_distances}")

Starting Bellman-Ford algorithm from node: A
Edge list: [('A', 'B', 4), ('A', 'C', 2), ('B', 'A', 4), ('B', 'C', 1), ('B', 'D', 5), ('C', 'A', 2), ('C', 'B', 1), ('C', 'D', 8), ('C', 'E', 10), ('D', 'B', 5), ('D', 'C', 8), ('D', 'E', 2), ('D', 'F', 6), ('E', 'C', 10), ('E', 'D', 2), ('E', 'F', 3), ('E', 'G', 1), ('F', 'D', 6), ('F', 'E', 3), ('F', 'G', 4), ('G', 'E', 1), ('G', 'F', 4)]
Step by step execution:

Iteration 1:
  Updated B: inf → 4 (via A)
  Updated C: inf → 2 (via A)
  Updated D: inf → 9 (via B)
  Updated B: 4 → 3 (via C)
  Updated E: inf → 12 (via C)
  Updated E: 12 → 11 (via D)
  Updated F: inf → 15 (via D)
  Updated F: 15 → 14 (via E)
  Updated G: inf → 12 (via E)
  Current distances: {'A': 0, 'B': 3, 'C': 2, 'D': 9, 'E': 11, 'F': 14, 'G': 12}

Iteration 2:
  Updated D: 9 → 8 (via B)
  Updated E: 11 → 10 (via D)
  Updated F: 14 → 13 (via E)
  Updated G: 12 → 11 (via E)
  Current distances: {'A': 0, 'B': 3, 'C': 2, 'D': 8, 'E': 10, 'F': 13, 'G': 11}

Iteration 3:
  No up

## Floyd-Warshall Algorithm - All-Pairs Shortest Paths

**Floyd-Warshall Algorithm** finds shortest paths between **all pairs of vertices** in a weighted graph. It can handle negative weights but not negative cycles.

**Algorithm Steps:**
1. Initialize distance matrix with edge weights (infinity for no edge)
2. Set diagonal elements to 0 (distance from vertex to itself)
3. For each intermediate vertex k:
   - For each pair of vertices (i,j):
     - If path through k is shorter, update distance[i][j]
4. Result: matrix with shortest distances between all vertex pairs

**Time Complexity:** O(V³) - always, regardless of edge count
**Space Complexity:** O(V²) for the distance matrix

**Key Properties:**
- **Dynamic Programming**: Considers all possible intermediate vertices
- **All-pairs solution**: Finds shortest paths between every vertex pair
- **Negative weight support**: Can handle negative edges (but not negative cycles)
- **Dense graphs**: More efficient than running Dijkstra from each vertex

**Use Cases:**
- **Network analysis**: Finding shortest paths between all node pairs
- **Game AI**: Precomputing distances for fast pathfinding queries
- **Social networks**: Analyzing connection strengths between all users
- **Transportation**: Route planning with multiple source-destination pairs

In [8]:
def floyd_warshall(graph):
    """
    Floyd-Warshall algorithm for finding shortest paths between all pairs of vertices
    Returns distance matrix and path reconstruction matrix
    """
    n = len(graph.nodes)
    
    # Initialize distance matrix with adjacency matrix
    distances = [[float('inf') for _ in range(n)] for _ in range(n)]
    next_node = [[None for _ in range(n)] for _ in range(n)]
    
    # Set up initial distances
    for i in range(n):
        distances[i][i] = 0  # Distance from node to itself is 0
    
    # Add existing edges
    for u in graph.adj_list:
        u_idx = graph.node_to_index[u]
        for v, weight in graph.adj_list[u]:
            v_idx = graph.node_to_index[v]
            distances[u_idx][v_idx] = weight
            next_node[u_idx][v_idx] = v_idx
    
    print("Starting Floyd-Warshall algorithm")
    print("Initial distance matrix:")
    print_matrix(distances, graph.nodes)
    
    # Main Floyd-Warshall algorithm
    for k in range(n):
        k_node = graph.nodes[k]
        print(f"\nIteration {k+1}: Using {k_node} as intermediate vertex")
        
        updates = []
        for i in range(n):
            for j in range(n):
                if distances[i][k] + distances[k][j] < distances[i][j]:
                    old_dist = distances[i][j]
                    distances[i][j] = distances[i][k] + distances[k][j]
                    next_node[i][j] = next_node[i][k]
                    
                    i_node = graph.nodes[i]
                    j_node = graph.nodes[j]
                    updates.append(f"{i_node}→{j_node}: {old_dist} → {distances[i][j]}")
        
        if updates:
            print(f"Updates: {', '.join(updates)}")
        else:
            print("No updates in this iteration")
        
        print("Current distance matrix:")
        print_matrix(distances, graph.nodes)
    
    return distances, next_node

def print_matrix(matrix, nodes):
    """Helper function to print matrix with node labels"""
    n = len(nodes)
    print("    ", end="")
    for node in nodes:
        print(f"{node:>6}", end="")
    print()
    
    for i in range(n):
        print(f"{nodes[i]:>2}: ", end="")
        for j in range(n):
            val = matrix[i][j]
            if val == float('inf'):
                print("    ∞", end="")
            else:
                print(f"{val:>5}", end="")
        print()

def reconstruct_floyd_path(next_matrix, graph, start, end):
    """Reconstruct path from start to end using Floyd-Warshall next matrix"""
    start_idx = graph.node_to_index[start]
    end_idx = graph.node_to_index[end]
    
    if next_matrix[start_idx][end_idx] is None:
        return []
    
    path = [start]
    current_idx = start_idx
    
    while current_idx != end_idx:
        current_idx = next_matrix[current_idx][end_idx]
        path.append(graph.nodes[current_idx])
    
    return path

# Run Floyd-Warshall algorithm
fw_distances, fw_next = floyd_warshall(graph)

print("\n=== Floyd-Warshall Results ===")
print("Final shortest distances matrix:")
print_matrix(fw_distances, graph.nodes)

print("\nShortest paths between all pairs:")
for i, start in enumerate(graph.nodes):
    for j, end in enumerate(graph.nodes):
        if i != j:
            distance = fw_distances[i][j]
            if distance != float('inf'):
                path = reconstruct_floyd_path(fw_next, graph, start, end)
                print(f"{start} → {end}: {' → '.join(path)} (weight: {distance})")
            else:
                print(f"{start} → {end}: No path exists")

Starting Floyd-Warshall algorithm
Initial distance matrix:
         A     B     C     D     E     F     G
 A:     0    4    2    ∞    ∞    ∞    ∞
 B:     4    0    1    5    ∞    ∞    ∞
 C:     2    1    0    8   10    ∞    ∞
 D:     ∞    5    8    0    2    6    ∞
 E:     ∞    ∞   10    2    0    3    1
 F:     ∞    ∞    ∞    6    3    0    4
 G:     ∞    ∞    ∞    ∞    1    4    0

Iteration 1: Using A as intermediate vertex
No updates in this iteration
Current distance matrix:
         A     B     C     D     E     F     G
 A:     0    4    2    ∞    ∞    ∞    ∞
 B:     4    0    1    5    ∞    ∞    ∞
 C:     2    1    0    8   10    ∞    ∞
 D:     ∞    5    8    0    2    6    ∞
 E:     ∞    ∞   10    2    0    3    1
 F:     ∞    ∞    ∞    6    3    0    4
 G:     ∞    ∞    ∞    ∞    1    4    0

Iteration 2: Using B as intermediate vertex
Updates: A→D: inf → 9, C→D: 8 → 6, D→A: inf → 9, D→C: 8 → 6
Current distance matrix:
         A     B     C     D     E     F     G
 A:     0  

## Minimum Spanning Tree (MST) - Prim's Algorithm

**Prim's Algorithm** finds the Minimum Spanning Tree of a connected weighted graph. An MST connects all vertices with the minimum total edge weight and no cycles.

**Algorithm Steps:**
1. Start with an arbitrary vertex, add it to MST
2. While MST doesn't include all vertices:
   - Find the minimum weight edge connecting MST to a non-MST vertex
   - Add this edge and vertex to MST
3. Result: Tree with V-1 edges and minimum total weight

**Time Complexity:** O((V + E) log V) with binary heap, O(V²) with array
**Space Complexity:** O(V) for priority queue and MST tracking

**Key Properties:**
- **Greedy approach**: Always picks the minimum weight edge to expand MST
- **Cut property**: MST contains the minimum weight edge crossing any cut
- **Unique solution**: If all edge weights are distinct, MST is unique
- **Connected graph required**: Algorithm assumes graph is connected

**Use Cases:**
- **Network design**: Connecting cities with minimum cable cost
- **Circuit design**: Minimum wiring in electronic circuits  
- **Clustering**: Grouping data points with minimum total distance
- **Approximation algorithms**: Basis for TSP approximation

In [9]:
def prims_algorithm(graph):
    """
    Prim's algorithm for finding Minimum Spanning Tree (MST)
    Returns MST edges, total weight, and step-by-step construction
    """
    if not graph.nodes:
        return [], 0
    
    # Start with first node
    start_node = graph.nodes[0]
    mst_vertices = {start_node}
    mst_edges = []
    total_weight = 0
    
    # Priority queue: (weight, from_vertex, to_vertex)
    pq = []
    
    # Add all edges from start node to priority queue
    for neighbor, weight in graph.adj_list[start_node]:
        heapq.heappush(pq, (weight, start_node, neighbor))
    
    print(f"Starting Prim's algorithm from node: {start_node}")
    print("Step by step MST construction:")
    
    step = 1
    while pq and len(mst_vertices) < len(graph.nodes):
        weight, from_vertex, to_vertex = heapq.heappop(pq)
        
        # Skip if both vertices are already in MST (would create cycle)
        if to_vertex in mst_vertices:
            continue
        
        # Add edge to MST
        mst_edges.append((from_vertex, to_vertex, weight))
        mst_vertices.add(to_vertex)
        total_weight += weight
        
        print(f"Step {step}: Add edge {from_vertex}-{to_vertex} (weight: {weight})")
        print(f"         MST vertices: {sorted(mst_vertices)}")
        print(f"         MST weight so far: {total_weight}")
        
        # Add all edges from new vertex to priority queue
        new_edges = []
        for neighbor, edge_weight in graph.adj_list[to_vertex]:
            if neighbor not in mst_vertices:
                heapq.heappush(pq, (edge_weight, to_vertex, neighbor))
                new_edges.append(f"{to_vertex}-{neighbor}({edge_weight})")
        
        if new_edges:
            print(f"         Added to queue: {', '.join(new_edges)}")
        
        step += 1
        print()
    
    return mst_edges, total_weight

# Run Prim's algorithm
prims_mst, prims_weight = prims_algorithm(graph)

print("=== Prim's Algorithm Results ===")
print(f"MST edges: {prims_mst}")
print(f"Total MST weight: {prims_weight}")
print(f"Number of edges in MST: {len(prims_mst)}")
print(f"Expected edges for {len(graph.nodes)} vertices: {len(graph.nodes) - 1}")

print("\nMST structure:")
for from_v, to_v, weight in prims_mst:
    print(f"  {from_v} ↔ {to_v} (weight: {weight})")

# Verify MST properties
print(f"\nMST Verification:")
print(f"✓ Connected: MST has {len(prims_mst)} edges for {len(graph.nodes)} vertices")
print(f"✓ Acyclic: Tree structure with V-1 edges")
print(f"✓ Minimum weight: {prims_weight} (Prim's algorithm guarantees optimality)")

Starting Prim's algorithm from node: A
Step by step MST construction:
Step 1: Add edge A-C (weight: 2)
         MST vertices: ['A', 'C']
         MST weight so far: 2
         Added to queue: C-B(1), C-D(8), C-E(10)

Step 2: Add edge C-B (weight: 1)
         MST vertices: ['A', 'B', 'C']
         MST weight so far: 3
         Added to queue: B-D(5)

Step 3: Add edge B-D (weight: 5)
         MST vertices: ['A', 'B', 'C', 'D']
         MST weight so far: 8
         Added to queue: D-E(2), D-F(6)

Step 4: Add edge D-E (weight: 2)
         MST vertices: ['A', 'B', 'C', 'D', 'E']
         MST weight so far: 10
         Added to queue: E-F(3), E-G(1)

Step 5: Add edge E-G (weight: 1)
         MST vertices: ['A', 'B', 'C', 'D', 'E', 'G']
         MST weight so far: 11
         Added to queue: G-F(4)

Step 6: Add edge E-F (weight: 3)
         MST vertices: ['A', 'B', 'C', 'D', 'E', 'F', 'G']
         MST weight so far: 14

=== Prim's Algorithm Results ===
MST edges: [('A', 'C', 2), ('C', 'B', 

## Minimum Spanning Tree (MST) - Kruskal's Algorithm

**Kruskal's Algorithm** finds the Minimum Spanning Tree by considering edges in increasing order of weight and using Union-Find to detect cycles.

**Algorithm Steps:**
1. Sort all edges by weight in ascending order
2. Initialize Union-Find data structure for cycle detection
3. For each edge in sorted order:
   - If edge connects different components (no cycle), add it to MST
   - Use Union-Find to merge components
4. Stop when MST has V-1 edges

**Time Complexity:** O(E log E) for sorting edges + O(E α(V)) for Union-Find operations
**Space Complexity:** O(V) for Union-Find structure

**Key Properties:**
- **Greedy approach**: Always picks the minimum weight edge that doesn't create a cycle
- **Edge-based**: Considers edges globally rather than growing from a vertex
- **Union-Find**: Efficient cycle detection using disjoint set data structure
- **Sorting dependent**: Performance depends on edge sorting

**Use Cases:**
- **Same as Prim's**: Network design, circuit layout, clustering
- **Better for sparse graphs**: When E << V², Kruskal's can be more efficient
- **Parallel processing**: Easier to parallelize than Prim's algorithm

In [10]:
class UnionFind:
    """Union-Find (Disjoint Set) data structure for cycle detection"""
    
    def __init__(self, vertices):
        self.parent = {v: v for v in vertices}
        self.rank = {v: 0 for v in vertices}
    
    def find(self, x):
        """Find root of x with path compression"""
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])  # Path compression
        return self.parent[x]
    
    def union(self, x, y):
        """Union two sets by rank"""
        root_x = self.find(x)
        root_y = self.find(y)
        
        if root_x == root_y:
            return False  # Already in same set (would create cycle)
        
        # Union by rank
        if self.rank[root_x] < self.rank[root_y]:
            self.parent[root_x] = root_y
        elif self.rank[root_x] > self.rank[root_y]:
            self.parent[root_y] = root_x
        else:
            self.parent[root_y] = root_x
            self.rank[root_x] += 1
        
        return True  # Successfully united

def kruskals_algorithm(graph):
    """
    Kruskal's algorithm for finding Minimum Spanning Tree (MST)
    Returns MST edges, total weight, and step-by-step construction
    """
    # Create edge list from adjacency list
    edge_list = []
    for u in graph.adj_list:
        for v, weight in graph.adj_list[u]:
            # Add each edge only once (u < v to avoid duplicates)
            if u < v:
                edge_list.append((weight, u, v))
    
    # Sort edges by weight
    edge_list.sort()
    
    print("Starting Kruskal's algorithm")
    print(f"All edges sorted by weight: {edge_list}")
    print("Step by step MST construction:")
    
    # Initialize Union-Find
    uf = UnionFind(graph.nodes)
    mst_edges = []
    total_weight = 0
    
    step = 1
    for weight, u, v in edge_list:
        print(f"Step {step}: Consider edge {u}-{v} (weight: {weight})")
        
        # Check if adding this edge creates a cycle
        if uf.union(u, v):
            # No cycle, add edge to MST
            mst_edges.append((u, v, weight))
            total_weight += weight
            
            print(f"         ✓ Added to MST (no cycle)")
            print(f"         Current MST edges: {len(mst_edges)}")
            print(f"         MST weight so far: {total_weight}")
            
            # Stop if MST is complete
            if len(mst_edges) == len(graph.nodes) - 1:
                print(f"         MST complete with {len(mst_edges)} edges")
                break
        else:
            # Would create cycle, reject edge
            print(f"         ✗ Rejected (would create cycle)")
        
        step += 1
        print()
    
    return mst_edges, total_weight

# Run Kruskal's algorithm
kruskals_mst, kruskals_weight = kruskals_algorithm(graph)

print("=== Kruskal's Algorithm Results ===")
print(f"MST edges: {kruskals_mst}")
print(f"Total MST weight: {kruskals_weight}")
print(f"Number of edges in MST: {len(kruskals_mst)}")

print("\nMST structure:")
for u, v, weight in kruskals_mst:
    print(f"  {u} ↔ {v} (weight: {weight})")

# Compare with Prim's result
print(f"\n=== MST Algorithm Comparison ===")
print(f"Prim's MST weight: {prims_weight}")
print(f"Kruskal's MST weight: {kruskals_weight}")
print(f"Both algorithms found same optimal weight: {prims_weight == kruskals_weight}")

# The actual edges might be different but weight should be the same
prims_edge_set = {tuple(sorted([u, v])) for u, v, w in prims_mst}
kruskals_edge_set = {tuple(sorted([u, v])) for u, v, w in kruskals_mst}
print(f"Same edge set: {prims_edge_set == kruskals_edge_set}")

Starting Kruskal's algorithm
All edges sorted by weight: [(1, 'B', 'C'), (1, 'E', 'G'), (2, 'A', 'C'), (2, 'D', 'E'), (3, 'E', 'F'), (4, 'A', 'B'), (4, 'F', 'G'), (5, 'B', 'D'), (6, 'D', 'F'), (8, 'C', 'D'), (10, 'C', 'E')]
Step by step MST construction:
Step 1: Consider edge B-C (weight: 1)
         ✓ Added to MST (no cycle)
         Current MST edges: 1
         MST weight so far: 1

Step 2: Consider edge E-G (weight: 1)
         ✓ Added to MST (no cycle)
         Current MST edges: 2
         MST weight so far: 2

Step 3: Consider edge A-C (weight: 2)
         ✓ Added to MST (no cycle)
         Current MST edges: 3
         MST weight so far: 4

Step 4: Consider edge D-E (weight: 2)
         ✓ Added to MST (no cycle)
         Current MST edges: 4
         MST weight so far: 6

Step 5: Consider edge E-F (weight: 3)
         ✓ Added to MST (no cycle)
         Current MST edges: 5
         MST weight so far: 9

Step 6: Consider edge A-B (weight: 4)
         ✗ Rejected (would create cyc

## Algorithm Complexity and Performance Analysis

Let's analyze the time and space complexity of all algorithms we've implemented:

### Time Complexity Summary

| Algorithm | Best Case | Average Case | Worst Case | Space Complexity |
|-----------|-----------|--------------|------------|------------------|
| **BFS** | O(V + E) | O(V + E) | O(V + E) | O(V) |
| **DFS** | O(V + E) | O(V + E) | O(V + E) | O(V) |
| **Dijkstra** | O((V + E) log V) | O((V + E) log V) | O((V + E) log V) | O(V) |
| **Bellman-Ford** | O(VE) | O(VE) | O(VE) | O(V) |
| **Floyd-Warshall** | O(V³) | O(V³) | O(V³) | O(V²) |
| **Prim's MST** | O((V + E) log V) | O((V + E) log V) | O((V + E) log V) | O(V) |
| **Kruskal's MST** | O(E log E) | O(E log E) | O(E log E) | O(V) |

### Graph Representation Complexity

| Representation | Space | Add Edge | Remove Edge | Check Edge | Get Neighbors |
|----------------|-------|----------|-------------|------------|---------------|
| **Adjacency Matrix** | O(V²) | O(1) | O(1) | O(1) | O(V) |
| **Adjacency List** | O(V + E) | O(1) | O(E) | O(E) | O(degree) |

### Performance Characteristics

**Dense Graphs (E ≈ V²):**
- **Adjacency Matrix** representation is more space-efficient
- **Floyd-Warshall** becomes competitive with repeated Dijkstra
- **Prim's** with adjacency matrix can be O(V²)

**Sparse Graphs (E << V²):**
- **Adjacency List** representation is preferred
- **Kruskal's** often outperforms Prim's
- **Dijkstra** is much faster than Bellman-Ford

**Special Cases:**
- **Negative weights**: Only Bellman-Ford and Floyd-Warshall work
- **All-pairs shortest paths**: Floyd-Warshall vs V runs of Dijkstra
- **Single-source shortest paths**: Dijkstra for non-negative, Bellman-Ford for negative

In [11]:
def performance_analysis():
    """
    Analyze the performance characteristics of our graph algorithms
    """
    V = len(graph.nodes)
    E = len(edges)
    
    print("=== Performance Analysis ===")
    print(f"Graph characteristics:")
    print(f"  Vertices (V): {V}")
    print(f"  Edges (E): {E}")
    print(f"  Graph density: {E / (V * (V-1) / 2):.2f} ({E}/{V*(V-1)//2} possible edges)")
    print(f"  Average degree: {2*E/V:.1f}")
    
    print(f"\nAlgorithm complexity for this graph:")
    
    # Calculate theoretical operations for each algorithm
    bfs_ops = V + E
    dfs_ops = V + E
    dijkstra_ops = (V + E) * 7  # log V ≈ 3 for 7 vertices, using 7 for simplicity
    bellman_ford_ops = V * E
    floyd_ops = V**3
    prims_ops = (V + E) * 7
    kruskals_ops = E * 4  # log E ≈ 4 for 11 edges
    
    print(f"  BFS/DFS: O(V + E) = O({V} + {E}) = {bfs_ops} operations")
    print(f"  Dijkstra: O((V + E) log V) ≈ {dijkstra_ops} operations")
    print(f"  Bellman-Ford: O(V × E) = {V} × {E} = {bellman_ford_ops} operations")
    print(f"  Floyd-Warshall: O(V³) = {V}³ = {floyd_ops} operations")
    print(f"  Prim's MST: O((V + E) log V) ≈ {prims_ops} operations")
    print(f"  Kruskal's MST: O(E log E) ≈ {kruskals_ops} operations")
    
    print(f"\nAlgorithm recommendations:")
    print(f"  Single-source shortest path (non-negative): Dijkstra (most efficient)")
    print(f"  Single-source shortest path (negative weights): Bellman-Ford (only option)")
    print(f"  All-pairs shortest paths: Floyd-Warshall vs {V} × Dijkstra")
    print(f"    Floyd-Warshall: {floyd_ops} operations")
    print(f"    {V} × Dijkstra: {V * dijkstra_ops} operations")
    
    if floyd_ops < V * dijkstra_ops:
        print(f"    → Floyd-Warshall is better for all-pairs")
    else:
        print(f"    → Multiple Dijkstra runs are better")
    
    print(f"  Minimum Spanning Tree: Kruskal's ({kruskals_ops}) vs Prim's ({prims_ops})")
    if kruskals_ops < prims_ops:
        print(f"    → Kruskal's is better for this graph")
    else:
        print(f"    → Prim's is better for this graph")

performance_analysis()

# Memory usage comparison
print(f"\n=== Memory Usage Comparison ===")
print(f"Adjacency Matrix: {len(graph.nodes)}² = {len(graph.nodes)**2} entries")
print(f"Adjacency List: {len(graph.nodes)} lists + {2*len(edges)} entries = {len(graph.nodes) + 2*len(edges)} total")

matrix_memory = len(graph.nodes)**2 * 4  # 4 bytes per integer
list_memory = len(graph.nodes) * 8 + 2*len(edges) * 12  # 8 bytes per list + 12 bytes per entry

print(f"Estimated memory (bytes):")
print(f"  Adjacency Matrix: {matrix_memory} bytes")
print(f"  Adjacency List: {list_memory} bytes")
print(f"  Better choice: {'Matrix' if matrix_memory < list_memory else 'List'}")

=== Performance Analysis ===
Graph characteristics:
  Vertices (V): 7
  Edges (E): 11
  Graph density: 0.52 (11/21 possible edges)
  Average degree: 3.1

Algorithm complexity for this graph:
  BFS/DFS: O(V + E) = O(7 + 11) = 18 operations
  Dijkstra: O((V + E) log V) ≈ 126 operations
  Bellman-Ford: O(V × E) = 7 × 11 = 77 operations
  Floyd-Warshall: O(V³) = 7³ = 343 operations
  Prim's MST: O((V + E) log V) ≈ 126 operations
  Kruskal's MST: O(E log E) ≈ 44 operations

Algorithm recommendations:
  Single-source shortest path (non-negative): Dijkstra (most efficient)
  Single-source shortest path (negative weights): Bellman-Ford (only option)
  All-pairs shortest paths: Floyd-Warshall vs 7 × Dijkstra
    Floyd-Warshall: 343 operations
    7 × Dijkstra: 882 operations
    → Floyd-Warshall is better for all-pairs
  Minimum Spanning Tree: Kruskal's (44) vs Prim's (126)
    → Kruskal's is better for this graph

=== Memory Usage Comparison ===
Adjacency Matrix: 7² = 49 entries
Adjacency List

## Summary and Comparison Table

Here's a comprehensive comparison of all the algorithms and techniques covered in this notebook:

### Algorithm Classification and Use Cases

| Category | Algorithm | Purpose | Key Strength | Limitation |
|----------|-----------|---------|--------------|------------|
| **Graph Traversal** | BFS | Level-order exploration | Shortest hop count | No edge weights |
| | DFS (Preorder) | Depth-first exploration | Memory efficient | May not find shortest path |
| | DFS (Postorder) | Topological ordering | Good for dependencies | Stack overflow risk |
| **Single-Source Shortest Path** | Dijkstra | Non-negative weights | Optimal + efficient | No negative weights |
| | Bellman-Ford | Negative weights allowed | Detects negative cycles | Slower O(VE) |
| **All-Pairs Shortest Path** | Floyd-Warshall | All vertex pairs | Simple implementation | O(V³) complexity |
| **Minimum Spanning Tree** | Prim's | Growing tree approach | Good for dense graphs | Requires connected graph |
| | Kruskal's | Edge-based approach | Good for sparse graphs | Requires sorting edges |

### Time Complexity Comparison

| Algorithm | Time Complexity | Space Complexity | Best For |
|-----------|----------------|------------------|----------|
| **BFS/DFS** | O(V + E) | O(V) | Graph traversal, connectivity |
| **Dijkstra** | O((V + E) log V) | O(V) | Single-source, non-negative weights |
| **Bellman-Ford** | O(V × E) | O(V) | Negative weights, cycle detection |
| **Floyd-Warshall** | O(V³) | O(V²) | All-pairs, dense graphs |
| **Prim's MST** | O((V + E) log V) | O(V) | MST, dense graphs |
| **Kruskal's MST** | O(E log E) | O(V) | MST, sparse graphs |

### Algorithm Selection Guide

**For Shortest Paths:**
- **Single source, non-negative weights** → Dijkstra's Algorithm
- **Single source, negative weights** → Bellman-Ford Algorithm  
- **All pairs, small graphs** → Floyd-Warshall Algorithm
- **All pairs, large graphs** → Multiple Dijkstra runs

**For Graph Traversal:**
- **Level-by-level exploration** → Breadth-First Search (BFS)
- **Deep exploration, backtracking** → Depth-First Search (DFS)
- **Topological ordering** → DFS Postorder

**For Minimum Spanning Tree:**
- **Dense graphs** → Prim's Algorithm
- **Sparse graphs** → Kruskal's Algorithm
- **Online/streaming edges** → Kruskal's Algorithm

**Graph Representation Choice:**
- **Dense graphs (E ≈ V²)** → Adjacency Matrix
- **Sparse graphs (E << V²)** → Adjacency List
- **Frequent edge lookups** → Adjacency Matrix
- **Memory constrained** → Adjacency List

This completes our comprehensive guide to Undirected Weighted Graphs and their fundamental algorithms!